# 全部训练代码

# gen_P12_classify_regression_data 

In [2]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
from core.tool import IoU
import numpy as np
from numpy.random import randint
import cv2
import os
import time

def main():

    f1 = open(os.path.join(save_dir, 'pos_%d.txt'%(img_size)), 'w')
    f2 = open(os.path.join(save_dir, 'neg_%d.txt'%(img_size)), 'w')
    f3 = open(os.path.join(save_dir, 'par_%d.txt'%(img_size)), 'w')    
    
    with open(WIDER_spilt_dir) as filenames:
        p=0
        neg_idx=0
        pos_idx=0
        par_idx=0
        for line in filenames.readlines():
            line=line.strip().split(' ')
            if(p==0):
                pic_dir=line[0]
                p=1
                boxes=[]
            elif(p==1):
                k=int(line[0])
                p=2
            elif(p==2):
                b=[]            
                k=k-1
                if(k==0):
                    p=0                
                for i in range(4):
                    b.append(int(line[i]))
                boxes.append(b)
                # format of boxes is [x,y,w,h]
                if(p==0):
                    img=cv2.imread(os.path.join(WIDER_dir,pic_dir))
                    h,w,c=img.shape
                    
                    #save num negative pics whose IoU less than 0.3
                    num=50
                    while(num):
                        size=randint(12,min(w,h)/2)
                        x=randint(0,w-size)
                        y=randint(0,h-size)
                        if(np.max(IoU(np.array([x,y,x+size,y+size]),np.array(boxes)))<0.3):
                            resized_img = cv2.resize(img[y:y+size,x:x+size,:], (img_size, img_size))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                            num=num-1       

                    for box in boxes:
                        if((box[0]<0)|(box[1]<0)|(max(box[2],box[3])<20)|(min(box[2],box[3])<=5)): 
                            continue  
                        x1, y1, w1, h1 = box
                        
                        # crop images near the bounding box if IoU less than 0.3, save as negative samples
                        for i in range(20):
                            size = randint(12, min(w, h) / 2)
                            delta_x = randint(max(-size, -x1), w1)
                            delta_y = randint(max(-size, -y1), h1)
                            nx1 = int(max(0, x1 + delta_x))
                            ny1 = int(max(0, y1 + delta_y))
                            if((nx1 + size > w1)|(ny1 + size > h1)):
                                continue
                            if(np.max(IoU(np.array([nx1,ny1,nx1+size,ny1+size]),np.array(boxes)))<0.3):
                                resized_img = cv2.resize(img[y:y+size,x:x+size,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                                f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                                neg_idx=neg_idx+1
                                
                        #save num positive&part face whose IoU more than 0.65|0.4         
                        box_ = np.array(box).reshape(1, -1)
                        for i in range(15):
                            size=randint(np.floor(0.8*min(w1,h1)),np.ceil(1.25*max(w1,h1))+1)
                        
                            delta_w = randint(-w1 * 0.2, w1 * 0.2 + 1)
                            delta_h = randint(-h1 * 0.2, h1 * 0.2 + 1)
                            # random face box
                            nx1 = int(max(x1 + w1 / 2 + delta_w - size / 2, 0))
                            ny1 = int(max(y1 + h1 / 2 + delta_h - size / 2, 0))
                            nx2 = nx1 + size
                            ny2 = ny1 + size
                            
                            if( nx2 > w | ny2 > h):
                                continue 
                                
                            offset_x1 = (x1 - nx1) / float(size)
                            offset_y1 = (y1 - ny1) / float(size)
                            offset_x2 = (x1+w1 - nx2) / float(size)
                            offset_y2 = (y1+h1 - ny2) / float(size)                                
    
                            if(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.65):                     
                                resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                                f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                                pos_idx=pos_idx+1   
                                
                            elif(IoU(np.array([nx1,ny1,nx2,ny2]),box_)>0.4):
                                resized_img = cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size))
                                cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                                f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                                par_idx=par_idx+1 
        print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))
        
    f1.close()
    f2.close()
    f3.close()  


if __name__=="__main__":
    
    img_size=12

    base_dir="/home/dell/Desktop/prepared_data"
    
    WIDER_dir=os.path.join(base_dir,"WIDER_train/images")
    WIDER_spilt_dir=os.path.join(base_dir,"wider_face_split/wider_face_train_bbx_gt.txt")
    negative_dir=os.path.join(base_dir,"DATA/%d/negative"%(img_size))
    positive_dir=os.path.join(base_dir,"DATA/%d/positive"%(img_size))
    par_dir=os.path.join(base_dir,"DATA/%d/part"%(img_size))
    save_dir=os.path.join(base_dir,"DATA/%d"%(img_size))
    
    if not os.path.exists(positive_dir):
        os.makedirs(positive_dir)
    if not os.path.exists(par_dir):
        os.makedirs(par_dir)
    if not os.path.exists(negative_dir):
        os.makedirs(negative_dir) 
        
    begin=time.time()
    
    main()

    print(time.time()-begin)
    
# pics all done,neg_pics 645895 in total,pos_pics 427415 in total,par_pics 761871 in total
# 706.3623492717743

pics all done,neg_pics 645895 in total,pos_pics 427415 in total,par_pics 761871 in total
706.3623492717743


# gen_P12_landmark_data

In [3]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""

from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

def main():
    
    f4 = open(os.path.join(save_dir, 'land_%d.txt'%(img_size)), 'w')

    with open(pic_spilt_dir) as filenames:
        land_idx=0
        for line in filenames:
            img_list=[]
            mark_list=[]
            line=line.strip().split(' ')
            img=cv2.imread(os.path.join(lfw_dir,line[0].replace("\\","/")))
            box=(line[1],line[2],line[3],line[4])
            box=[int(_) for _ in box]
            #format of box is [x,x+w,y,y+h]
            height,weight,channel=img.shape
            landmark=np.zeros((5,2))
            for i in range(5):
                mark=(float(line[5+2*i]),float(line[5+2*i+1]))
                landmark[i]=mark

            facemark=np.zeros((5,2))
            for i in range(5):
                mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
                facemark[i]=mark
            img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (img_size, img_size)))  
            mark_list.append(facemark.reshape(10))

            box_=[box[0],box[2],box[1],box[3]]
            #format of box is [x,y,x+w,y+h]      
            x1,y1,x2,y2=box_
            w=x2-x1+1
            h=y2-y1+1

            if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
                continue          
            num=50
            while(num):

                size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)

                delta_w = randint(-w * 0.2, w * 0.2 + 1)
                delta_h = randint(-h * 0.2, h * 0.2 + 1)
                # random face box
                nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
                ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
                nx2 = nx1 + size
                ny2 = ny1 + size 

                if( nx2 > weight | ny2 > height):
                    continue               

                _box=[x1,y1,w,h]
                _box=np.array(_box).reshape(1,-1)     
                if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                    facemark=np.zeros((5,2))
                    for i in range(5):
                        mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                        facemark[i]=mark  
                    img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size)))  
                    mark_list.append(facemark.reshape(10))

                    #mirro
                    mirro_mark=facemark.copy()
                    if(random.choice([0,1])):
                        img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                        img_list.append(cv2.resize(img1, (img_size, img_size)))  
                        mark_list.append(mirro_mark.reshape(10))  

                    num=num-1
            for i in range(len(img_list)):

                if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                    continue

                if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                    continue

                cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
                mark=[str(_)for _ in mark_list[i]]
                f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
                land_idx=land_idx+1
            
        print("pics all done,land_pics %d in total"%(land_idx))
        
    f4.close()    

if __name__=="__main__":

    img_size=12
    
    #change img_size to P=12 R=24 O=48 net
    
    begin=time.time()
    
    base_dir="/home/dell/Desktop/prepared_data"
    
    lfw_dir=os.path.join(base_dir,"train")
    pic_spilt_dir=os.path.join(base_dir,"train/trainImageList.txt")
    landmark_dir=os.path.join(base_dir,"DATA/%d/landmark"%(img_size))
    save_dir=os.path.join(base_dir,"DATA/%d"%(img_size))  

    if not os.path.exists(landmark_dir):
        os.makedirs(landmark_dir)    

    main()
    
    print(time.time()-begin)
    
# pics all done,land_pics 727571 in total
# 240.12906050682068

pics all done,land_pics 727571 in total
240.12906050682068


# gen_P12_tfrecord

In [5]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf 
import cv2
import random
import time
import os

def main():
    
    t_time=time.time()   
    for index,term in enumerate(terms):
        num=0
        print("%s start"%(term))
        tfr_addr=os.path.join(base_dir,"DATA/%d/%s_train.tfrecords"%(img_size,term))
        with tf.python_io.TFRecordWriter(tfr_addr) as writer:
            file_addr=os.path.join(base_dir,"DATA/%d/%s.txt"%(img_size,term))
            with open(file_addr) as readlines:
                readlines=[line.strip().split(' ') for line in readlines]
                random.shuffle(readlines)
                for i,line in enumerate(readlines):
                    if(num%10000==0):
                        print(i,time.time()-t_time)
                        t_time=time.time()
                    img=cv2.imread(line[0])
                    if(img is None):
                        continue
                    img_raw = img.tobytes()
                    label=int(line[1])
                    roi=[0.0]*4               
                    landmark=[0.0]*10
                    if(len(line)==6):    
                        roi=[float(_) for _ in line[2:6]]   
                    if(len(line)==12):
                        landmark=[float(_) for _ in line[2:12]]                  
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                        "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                    }))
                    writer.write(example.SerializeToString())  #序列化为字符串  
                    num+=1
                    if(num==base*scale[index]):
                        print("%s finish"%(term))
                        break

if __name__=="__main__":
    
    img_size=12
    #change img_size to P=12 R=24 O=48 net
    terms=['neg_%d'%(img_size),'pos_%d'%(img_size),'par_%d'%(img_size),'land_%d'%(img_size)]
    scale=[3,1,1,2]
    
    base_dir="/home/dell/Desktop/prepared_data"

    #set base number of pos_pic    
    base=200000

    begin=time.time()

    main()
    
    print(time.time()-begin)
    
# 2244.9860048294067

neg_12 start
0 1.8470921516418457
10000 7.4005208015441895
20000 10.426478862762451
30000 27.678472995758057
40000 15.575403928756714
50000 24.907623291015625
60000 27.819153308868408
70000 8.80367660522461
80000 8.094653129577637
90000 28.95293927192688
100000 21.377321004867554
110000 44.27895188331604
120000 15.256330013275146
130000 22.440184593200684
140000 8.09855604171753
150000 16.776840448379517
160000 31.46169948577881
170000 19.120004892349243
180000 8.865959167480469
190000 17.430410861968994
200000 26.139073610305786
210000 21.776042461395264
220000 17.368127822875977
230000 8.614549398422241
240000 16.177074670791626
250000 25.828405380249023
260000 31.22150206565857
270000 8.651751279830933
280000 7.871500253677368
290000 51.32798957824707
300000 35.24047875404358
310000 27.555583238601685
320000 19.57759118080139
330000 8.052394390106201
340000 7.4418182373046875
350000 27.920541286468506
360000 9.918498992919922
370000 7.845425367355347
380000 27.109370231628418
390000

# train_P12_model

In [1]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf
import time
from core.tool import read_multi_tfrecords,image_color_distort
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from train.train_tool import label_los,roi_los,landmark_los,cal_accuracy
import os

def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image,batch)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los*ratio[0]+_box_los*ratio[1]+_landmark_los*ratio[2]

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 

    opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy,truth_accuracy,false_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 
        tf.summary.scalar('ture_accuracy',truth_accuracy) 
        tf.summary.scalar('false_accuracy',false_accuracy) 
        
    saver=tf.train.Saver(max_to_keep=10)
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
        
        writer_train=tf.summary.FileWriter(os.path.join(base_dir,"model/%s/"%(model_name)),sess.graph)
        try:
            
            for i in range(1,train_step):
                
                image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
                
                sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                if(i%100==0):
                    summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                    writer_train.add_summary(summary,i) 
                if(i%1000==0):
                    print('次数',i)    
                    print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('truth_accuracy',sess.run(truth_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('false_accuracy',sess.run(false_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})) 
                    print('_label_los',sess.run(_label_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_box_los',sess.run(_box_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_landmark_los',sess.run(_landmark_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('time',time.time()-begin)
                    if(i%10000==0):
                        saver.save(sess,os.path.join(base_dir,"model/%s/%s.ckpt"%(model_name,model_name)),global_step=i)
        except  tf.errors.OutOfRangeError:
            print("finished")
        finally:
            coord.request_stop()
            writer_train.close()
        coord.join(threads)
    
def main(model):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,[batch,img_size,img_size,3],name='image')
        label=tf.placeholder(tf.int32,[batch],name='label')
        roi=tf.placeholder(tf.float32,[batch,4],name='roi')
        landmark = tf.placeholder(tf.float32,[batch,10],name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    base_dir="/home/dell/Desktop/prepared_data"
    img_size=12
    batch=448
    batch_size=[192,64,64,128]
    
    addr=[os.path.join(base_dir,"DATA/%d/neg_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/pos_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/par_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/land_%d_train.tfrecords"%(img_size,img_size))]  

    model=Pnet_model
    model_name="Pnet_model"    
    train_step=50001
    learning_rate=0.001
    
    save_model_path=os.path.join(base_dir,"model/%s"%(model_name))
    
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path) 
        
    if(model_name=="Onet_model"):
        ratio=[1,0.5,1]
    else:
        ratio=[1,0.5,0.5]
    

    begin=time.time()        
    main(model)
    
# tensorboard --logdir=/home/dell/Desktop/prepared_data/model/Pnet_model/

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


次数 1000
train_accuracy 0.85546875
ture_accuracy 0.703125
false_accuracy 0.90625
loss 0.4790472
_label_los 0.4376981
_box_los 0.030132672
_landmark_los 0.024308765
time 38.65703988075256
次数 2000
train_accuracy 0.890625
ture_accuracy 0.71875
false_accuracy 0.9479167
loss 0.39563435
_label_los 0.36148417
_box_los 0.022132492
_landmark_los 0.018500537
time 66.69461107254028
次数 3000
train_accuracy 0.953125
ture_accuracy 0.828125
false_accuracy 0.9947917
loss 0.2935558
_label_los 0.2602204
_box_los 0.02143247
_landmark_los 0.017962014
time 94.44382882118225
次数 4000
train_accuracy 0.921875
ture_accuracy 0.75
false_accuracy 0.9791667
loss 0.35046113
_label_los 0.31731704
_box_los 0.024801156
_landmark_los 0.013702227
time 122.52346277236938
次数 5000
train_accuracy 0.91015625
ture_accuracy 0.796875
false_accuracy 0.9479167
loss 0.37290937
_label_los 0.34031934
_box_los 0.021318134
_landmark_los 0.015247461
time 150.74694347381592
次数 6000
train_accuracy 0.92578125
ture_accuracy 0.78125
false_accu

次数 45000
train_accuracy 0.94140625
ture_accuracy 0.8125
false_accuracy 0.984375
loss 0.2618106
_label_los 0.2246515
_box_los 0.024082372
_landmark_los 0.0116351945
time 1274.5746665000916
次数 46000
train_accuracy 0.91796875
ture_accuracy 0.78125
false_accuracy 0.9635417
loss 0.2998927
_label_los 0.26507524
_box_los 0.020310273
_landmark_los 0.010545232
time 1302.64031457901
次数 47000
train_accuracy 0.93359375
ture_accuracy 0.90625
false_accuracy 0.9427083
loss 0.2866599
_label_los 0.25388497
_box_los 0.016882392
_landmark_los 0.009593657
time 1330.5746376514435
次数 48000
train_accuracy 0.9453125
ture_accuracy 0.828125
false_accuracy 0.984375
loss 0.2647419
_label_los 0.2284468
_box_los 0.019780474
_landmark_los 0.013806335
time 1358.4712142944336
次数 49000
train_accuracy 0.9375
ture_accuracy 0.796875
false_accuracy 0.984375
loss 0.2961814
_label_los 0.26010922
_box_los 0.020859893
_landmark_los 0.01209279
time 1386.3692576885223
次数 50000
train_accuracy 0.921875
ture_accuracy 0.796875
false

# gen_R24_hard_sample

In [3]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
from detection.mtcnn_detector import MTCNN_Detector
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from core.tool import IoU
import cv2
import numpy as np
import os
import time

def main():
    
    if(model_name in ["Pnet","Rnet","Onet"]):
        model[0]=Pnet_model
    if(model_name in ["Rnet","Onet"]):
        model[1]=Rnet_model
    if(model_name=="Onet"):
        model[2]=Onet_model
        
    detector=MTCNN_Detector(model,model_path,batch_size,factor,min_face_size,threshold)

    with open(WIDER_spilt_dir) as filenames:
        p=0
        neg_idx=0
        pos_idx=0
        par_idx=0
        idx=0
        for line in filenames.readlines():
            line=line.strip().split(' ')
            if(p==0):
                pic_dir=line[0]
                p=1
                boxes=[]
            elif(p==1):
                k=int(line[0])
                p=2
            elif(p==2):
                b=[]            
                k=k-1
                if(k==0):
                    p=0                
                for i in range(4):
                    b.append(int(line[i]))
                boxes.append(b)
                # format of boxes is [x,y,w,h]
                if(p==0):

                    img=cv2.imread(os.path.join(WIDER_dir,pic_dir))

                    face_box,_=detector.detect_single_face(img)
                    neg_num=0
                    for t_box in face_box:

                        ti_box=t_box.copy()
                        ti_box=[int(_) for _ in ti_box]   
                        
                        Iou=IoU(np.array(t_box),np.array(boxes))

                        if((np.max(Iou)<0.3)&(neg_num<60)):
                            resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size,img_size))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                            neg_num=neg_num+1

                        else:
                            x1,y1,w1,h1=boxes[np.argmax(Iou)]

                            offset_x1 = (x1 - t_box[0]) / float(t_box[2]-t_box[0]+1)
                            offset_y1 = (y1 - t_box[1]) / float(t_box[3]-t_box[1]+1)
                            offset_x2 = (x1+w1 - t_box[2]) / float(t_box[2]-t_box[0]+1)
                            offset_y2 = (y1+h1 - t_box[3]) / float(t_box[3]-t_box[1]+1)                         

                            if(np.max(Iou)>0.65):                    
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                                f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                                pos_idx=pos_idx+1   

                            elif(np.max(Iou)>0.4):
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                                f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                                par_idx=par_idx+1
                    idx+=1
                    if(idx%100==0):
                        print('idx: ',idx," ;neg_idx: ",neg_idx," ;pos_idx: ",pos_idx," ;par_idx: ",par_idx)
                        print(time.time()-begin)
    print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))

if __name__=="__main__":
    
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   
    base_dir="/home/dell/Desktop/prepared_data"
    #可修改参数
    threshold=[0.8,0.8,0.6]
    min_face_size=10
    
    #可修改参数
    batch_size=1
    img_size=24
    model_name="Pnet"    
    
    model_path=[os.path.join(base_dir,"model/Pnet_model/Pnet_model.ckpt-20000"),
                None,
                None] 
    #可修改参数
    
    factor=0.79
    model=[None,None,None]
    
    WIDER_dir=os.path.join(base_dir,"WIDER_train/images")
    WIDER_spilt_dir=os.path.join(base_dir,"wider_face_split/wider_face_train_bbx_gt.txt")
    negative_dir=os.path.join(base_dir,"DATA/%d/negative"%(img_size))
    positive_dir=os.path.join(base_dir,"DATA/%d/positive"%(img_size))
    par_dir=os.path.join(base_dir,"DATA/%d/part"%(img_size))
    save_dir=os.path.join(base_dir,"DATA/%d"%(img_size))
    
    if not os.path.exists(positive_dir):
        os.makedirs(positive_dir)
    if not os.path.exists(par_dir):
        os.makedirs(par_dir)
    if not os.path.exists(negative_dir):
        os.makedirs(negative_dir)    
    
    begin=time.time()
    
    f1 = open(os.path.join(save_dir, 'pos_%d.txt'%(img_size)), 'w')
    f2 = open(os.path.join(save_dir, 'neg_%d.txt'%(img_size)), 'w')
    f3 = open(os.path.join(save_dir, 'par_%d.txt'%(img_size)), 'w') 
    
    main()
    
    f1.close()
    f2.close()
    f3.close()      

    print(time.time()-begin)    
# pics all done,neg_pics 771431 in total,pos_pics 324412 in total,par_pics 866521 in total
# 64845.51023721695

# gen_R24_landmark_data

In [3]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

def main():
    
    f4 = open(os.path.join(save_dir, 'land_%d.txt'%(img_size)), 'w')

    with open(pic_spilt_dir) as filenames:
        land_idx=0
        for line in filenames:
            img_list=[]
            mark_list=[]
            line=line.strip().split(' ')
            img=cv2.imread(os.path.join(lfw_dir,line[0].replace("\\","/")))
            box=(line[1],line[2],line[3],line[4])
            box=[int(_) for _ in box]
            #format of box is [x,x+w,y,y+h]
            height,weight,channel=img.shape
            landmark=np.zeros((5,2))
            for i in range(5):
                mark=(float(line[5+2*i]),float(line[5+2*i+1]))
                landmark[i]=mark

            facemark=np.zeros((5,2))
            for i in range(5):
                mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
                facemark[i]=mark
            img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (img_size, img_size)))  
            mark_list.append(facemark.reshape(10))

            box_=[box[0],box[2],box[1],box[3]]
            #format of box is [x,y,x+w,y+h]      
            x1,y1,x2,y2=box_
            w=x2-x1+1
            h=y2-y1+1

            if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
                continue          
            num=30
            while(num):

                size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)

                delta_w = randint(-w * 0.2, w * 0.2 + 1)
                delta_h = randint(-h * 0.2, h * 0.2 + 1)
                # random face box
                nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
                ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
                nx2 = nx1 + size
                ny2 = ny1 + size 

                if( nx2 > weight | ny2 > height):
                    continue               

                _box=[x1,y1,w,h]
                _box=np.array(_box).reshape(1,-1)     
                if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                    facemark=np.zeros((5,2))
                    for i in range(5):
                        mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                        facemark[i]=mark  
                    img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size)))  
                    mark_list.append(facemark.reshape(10))

                    #mirro
                    mirro_mark=facemark.copy()
                    if(random.choice([0,1])):
                        img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                        img_list.append(cv2.resize(img1, (img_size, img_size)))  
                        mark_list.append(mirro_mark.reshape(10))  

                    num=num-1
            for i in range(len(img_list)):

                if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                    continue

                if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                    continue

                cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
                mark=[str(_)for _ in mark_list[i]]
                f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
                land_idx=land_idx+1
            
        print("pics all done,land_pics %d in total"%(land_idx))
        
    f4.close()    

if __name__=="__main__":

    img_size=24
    
    #change img_size to P=12 R=24 O=48 net
    
    begin=time.time()
    
    lfw_dir="/home/dell/Desktop/prepared_data/train"
    pic_spilt_dir="/home/dell/Desktop/prepared_data/train/trainImageList.txt"
    landmark_dir="/home/dell/Desktop/prepared_data/DATA/%d/landmark"%(img_size)
    save_dir="/home/dell/Desktop/prepared_data/DATA/%d"%(img_size)

    if not os.path.exists(landmark_dir):
        os.makedirs(landmark_dir)    

    main()
    
    print(time.time()-begin)
    
# pics all done,land_pics 440459 in total
# 116.04699325561523

pics all done,land_pics 440459 in total
116.04699325561523


# gen_R24_tfrecord

In [4]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf 
import cv2
import random
import time
import os

def main():
    
    t_time=time.time()   
    for index,term in enumerate(terms):
        num=0
        print("%s start"%(term))
        with tf.python_io.TFRecordWriter(os.path.join(base_dir,"DATA/%d/%s_train.tfrecords"%(img_size,term))) as writer:
            with open(os.path.join(base_dir,"DATA/%d/%s.txt"%(img_size,term))) as readlines:
                readlines=[line.strip().split(' ') for line in readlines]
                random.shuffle(readlines)
                for i,line in enumerate(readlines):
                    if(num%10000==0):
                        print(i,time.time()-t_time)
                        t_time=time.time()
                    #if windows
                    img=cv2.imread(line[0])
                    if(img is None):
                        continue
                    img_raw = img.tobytes()
                    label=int(line[1])
                    roi=[0.0]*4               
                    landmark=[0.0]*10
                    if(len(line)==6):    
                        roi=[float(_) for _ in line[2:6]]   
                    if(len(line)==12):
                        landmark=[float(_) for _ in line[2:12]]                  
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                        "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                    }))
                    writer.write(example.SerializeToString())  #序列化为字符串  
                    num+=1
                    if(num==base*scale[index]):
                        print("%s finish"%(term))
                        break

if __name__=="__main__":
    
    img_size=24
    #change img_size to P=12 R=24 O=48 net
    terms=['neg_%d'%(img_size),'pos_%d'%(img_size),'par_%d'%(img_size),'land_%d'%(img_size)]
    scale=[3,1,1,2]
    
    base_dir="/home/dell/Desktop/prepared_data"
    
    #set base number of pos_pic    
    base=220000

    begin=time.time()

    main()
    
    print(time.time()-begin)

neg_24 start
0 2.244324207305908
10000 13.05106496810913
20000 11.054393529891968
30000 16.148019552230835
40000 10.916179180145264
50000 14.674774646759033
60000 20.13191556930542
70000 10.43515658378601
80000 17.362887620925903
90000 11.11237359046936
100000 10.085326910018921
110000 17.9523868560791
120000 10.522304773330688
130000 11.979403495788574
140000 18.288910627365112
150000 11.179146528244019
160000 17.474924087524414
170000 11.464037895202637
180000 13.735122919082642
190000 14.726309776306152
200000 10.925694704055786
210000 17.144050359725952
220000 16.8756844997406
230000 10.424171686172485
240000 16.046016454696655
250000 21.672700881958008
260000 12.473885536193848
270000 20.5424063205719
280000 13.197590351104736
290000 11.718770503997803
300000 23.269935369491577
310000 11.533824920654297
320000 10.08726954460144
330000 20.09157419204712
340000 17.854693174362183
350000 20.249940395355225
360000 10.35850977897644
370000 10.699944972991943
380000 17.390256643295288
3

# train_R24_model

In [1]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf
import time
from core.tool import read_multi_tfrecords,image_color_distort
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from train.train_tool import label_los,roi_los,landmark_los,cal_accuracy
import os

def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image,batch)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los*ratio[0]+_box_los*ratio[1]+_landmark_los*ratio[2]

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 

    opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy,truth_accuracy,false_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 
        tf.summary.scalar('ture_accuracy',truth_accuracy) 
        tf.summary.scalar('false_accuracy',false_accuracy) 
        
    saver=tf.train.Saver(max_to_keep=10)
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
        
        writer_train=tf.summary.FileWriter(os.path.join(base_dir,"model/%s/"%(model_name)),sess.graph)
        try:
            
            for i in range(1,train_step):
                
                image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
                
                sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                if(i%100==0):
                    summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                    writer_train.add_summary(summary,i) 
                if(i%1000==0):
                    print('次数',i)    
                    print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('truth_accuracy',sess.run(truth_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('false_accuracy',sess.run(false_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})) 
                    print('_label_los',sess.run(_label_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_box_los',sess.run(_box_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_landmark_los',sess.run(_landmark_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('time',time.time()-begin)
                    if(i%10000==0):
                        saver.save(sess,os.path.join(base_dir,"model/%s/%s.ckpt"%(model_name,model_name)),global_step=i)
        except  tf.errors.OutOfRangeError:
            print("finished")
        finally:
            coord.request_stop()
            writer_train.close()
        coord.join(threads)
    
def main(model):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,[batch,img_size,img_size,3],name='image')
        label=tf.placeholder(tf.int32,[batch],name='label')
        roi=tf.placeholder(tf.float32,[batch,4],name='roi')
        landmark = tf.placeholder(tf.float32,[batch,10],name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    base_dir="/home/dell/Desktop/prepared_data"
    img_size=24
    batch=448
    batch_size=[192,64,64,128]
    
    addr=[os.path.join(base_dir,"DATA/%d/neg_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/pos_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/par_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/land_%d_train.tfrecords"%(img_size,img_size))]  

    model=Rnet_model
    model_name="Rnet_model"    
    train_step=50001
    learning_rate=0.001
    
    save_model_path=os.path.join(base_dir,"model/%s"%(model_name))
    
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path) 
        
    if(model_name=="Onet_model"):
        ratio=[1,0.5,1]
    else:
        ratio=[1,0.5,0.5]
    

    begin=time.time()        
    main(model)
    
# tensorboard --logdir=/home/dell/Desktop/prepared_data/model/Rnet_model/

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


次数 1000
train_accuracy 0.84765625
truth_accuracy 0.40625
false_accuracy 0.9947917
loss 0.57724446
_label_los 0.49139404
_box_los 0.031357292
_landmark_los 0.013418938
time 184.59069919586182
次数 2000
train_accuracy 0.87890625
truth_accuracy 0.671875
false_accuracy 0.9479167
loss 0.46575266
_label_los 0.390924
_box_los 0.032493345
_landmark_los 0.01014285
time 355.624986410141
次数 3000
train_accuracy 0.91015625
truth_accuracy 0.734375
false_accuracy 0.96875
loss 0.40624574
_label_los 0.33548185
_box_los 0.027392933
_landmark_los 0.008420416
time 526.2601375579834
次数 4000
train_accuracy 0.890625
truth_accuracy 0.71875
false_accuracy 0.9479167
loss 0.4732586
_label_los 0.40450013
_box_los 0.025242673
_landmark_los 0.00657386
time 695.461213350296
次数 5000
train_accuracy 0.9296875
truth_accuracy 0.84375
false_accuracy 0.9583333
loss 0.31908596
_label_los 0.25256535
_box_los 0.021942934
_landmark_los 0.006443594
time 866.5225236415863
次数 6000
train_accuracy 0.94140625
truth_accuracy 0.828125
f

_box_los 0.028323336
_landmark_los 0.004704252
time 7369.212498426437
次数 45000
train_accuracy 0.921875
truth_accuracy 0.765625
false_accuracy 0.9739583
loss 0.31670955
_label_los 0.24759996
_box_los 0.024979431
_landmark_los 0.0049622417
time 7538.784259796143
次数 46000
train_accuracy 0.91796875
truth_accuracy 0.71875
false_accuracy 0.984375
loss 0.38602918
_label_los 0.31843778
_box_los 0.023215666
_landmark_los 0.005351253
time 7705.519150257111
次数 47000
train_accuracy 0.94921875
truth_accuracy 0.859375
false_accuracy 0.9791667
loss 0.31394532
_label_los 0.24456325
_box_los 0.026794646
_landmark_los 0.004465622
time 7873.661741733551
次数 48000
train_accuracy 0.92578125
truth_accuracy 0.796875
false_accuracy 0.96875
loss 0.3369941
_label_los 0.2697218
_box_los 0.022253886
_landmark_los 0.0047117276
time 8041.7002811431885
次数 49000
train_accuracy 0.9140625
truth_accuracy 0.734375
false_accuracy 0.9739583
loss 0.32853195
_label_los 0.25872636
_box_los 0.028458174
_landmark_los 0.004415740

# gen_O48_hard_sample

In [5]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
from detection.mtcnn_detector import MTCNN_Detector
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from core.tool import IoU
import cv2
import numpy as np
import os
import time

def main():
    
    if(model_name in ["Pnet","Rnet","Onet"]):
        model[0]=Pnet_model
    if(model_name in ["Rnet","Onet"]):
        model[1]=Rnet_model
    if(model_name=="Onet"):
        model[2]=Onet_model
        
    detector=MTCNN_Detector(model,model_path,batch_size,factor,min_face_size,threshold)

    with open(WIDER_spilt_dir) as filenames:
        p=0
        neg_idx=0
        pos_idx=0
        par_idx=0
        idx=0
        for line in filenames.readlines():
            line=line.strip().split(' ')
            if(p==0):
                pic_dir=line[0]
                p=1
                boxes=[]
            elif(p==1):
                k=int(line[0])
                p=2
            elif(p==2):
                b=[]            
                k=k-1
                if(k==0):
                    p=0                
                for i in range(4):
                    b.append(int(line[i]))
                boxes.append(b)
                # format of boxes is [x,y,w,h]
                if(p==0):

                    img=cv2.imread(os.path.join(WIDER_dir,pic_dir))

                    face_box,_=detector.detect_single_face(img)
                    neg_num=0
                    for t_box in face_box:

                        ti_box=t_box.copy()
                        ti_box=[int(_) for _ in ti_box]   
                        
                        Iou=IoU(np.array(t_box),np.array(boxes))

                        if((np.max(Iou)<0.3)&(neg_num<60)):
                            resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size,img_size))
                            cv2.imwrite(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)),resized_img)
                            f2.write(os.path.join(negative_dir,'neg_%d.jpg'%(neg_idx)) + ' 0\n')
                            neg_idx=neg_idx+1
                            neg_num=neg_num+1

                        else:
                            x1,y1,w1,h1=boxes[np.argmax(Iou)]

                            offset_x1 = (x1 - t_box[0]) / float(t_box[2]-t_box[0]+1)
                            offset_y1 = (y1 - t_box[1]) / float(t_box[3]-t_box[1]+1)
                            offset_x2 = (x1+w1 - t_box[2]) / float(t_box[2]-t_box[0]+1)
                            offset_y2 = (y1+h1 - t_box[3]) / float(t_box[3]-t_box[1]+1)                         

                            if(np.max(Iou)>0.65):                    
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)),resized_img)
                                f1.write(os.path.join(positive_dir,'pos_%d.jpg'%(pos_idx)) + ' 1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))
                                pos_idx=pos_idx+1   

                            elif(np.max(Iou)>0.4):
                                resized_img = cv2.resize(img[ti_box[1]:ti_box[3],ti_box[0]:ti_box[2],:], (img_size, img_size))
                                cv2.imwrite(os.path.join(par_dir,'par_%d.jpg'%(par_idx)),resized_img)
                                f3.write(os.path.join(par_dir,'par_%d.jpg'%(par_idx)) + ' -1 %.2f %.2f %.2f %.2f\n'%(offset_x1,offset_y1,offset_x2,offset_y2))                           
                                par_idx=par_idx+1
                    idx+=1
                    if(idx%100==0):
                        print('idx: ',idx," ;neg_idx: ",neg_idx," ;pos_idx: ",pos_idx," ;par_idx: ",par_idx)
                        print(time.time()-begin)
    print("pics all done,neg_pics %d in total,pos_pics %d in total,par_pics %d in total"%(neg_idx,pos_idx,par_idx))

if __name__=="__main__":
    
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   
    base_dir="/home/dell/Desktop/prepared_data"
    #可修改参数
    threshold=[0.8,0.8,0.6]
    min_face_size=10
    
    #可修改参数
    batch_size=1
    img_size=48
    model_name="Rnet"    
    
    model_path=[os.path.join(base_dir,"model/Pnet_model/Pnet_model.ckpt-20000"),
                os.path.join(base_dir,"model/Rnet_model/Rnet_model.ckpt-40000"),
                None] 
    #可修改参数
    
    factor=0.79
    model=[None,None,None]
    
    WIDER_dir=os.path.join(base_dir,"WIDER_train/images")
    WIDER_spilt_dir=os.path.join(base_dir,"wider_face_split/wider_face_train_bbx_gt.txt")
    negative_dir=os.path.join(base_dir,"DATA/%d/negative"%(img_size))
    positive_dir=os.path.join(base_dir,"DATA/%d/positive"%(img_size))
    par_dir=os.path.join(base_dir,"DATA/%d/part"%(img_size))
    save_dir=os.path.join(base_dir,"DATA/%d"%(img_size))
    
    if not os.path.exists(positive_dir):
        os.makedirs(positive_dir)
    if not os.path.exists(par_dir):
        os.makedirs(par_dir)
    if not os.path.exists(negative_dir):
        os.makedirs(negative_dir)    
    
    begin=time.time()
    
    f1 = open(os.path.join(save_dir, 'pos_%d.txt'%(img_size)), 'w')
    f2 = open(os.path.join(save_dir, 'neg_%d.txt'%(img_size)), 'w')
    f3 = open(os.path.join(save_dir, 'par_%d.txt'%(img_size)), 'w') 
    
    main()
    
    f1.close()
    f2.close()
    f3.close()      

    print(time.time()-begin)    
# pics all done,neg_pics 622181 in total,pos_pics 228541 in total,par_pics 145649 in total
# 79577.55362534523

INFO:tensorflow:Restoring parameters from /home/dell/Desktop/prepared_data/model/Pnet_model/Pnet_model.ckpt-20000
INFO:tensorflow:Restoring parameters from /home/dell/Desktop/prepared_data/model/Rnet_model/Rnet_model.ckpt-40000
detect-time:  5.645401239395142
detect-time:  2.1593222618103027
detect-time:  2.7827980518341064
detect-time:  4.736136198043823
detect-time:  13.852707862854004
detect-time:  7.421629905700684
detect-time:  19.059978008270264
detect-time:  4.411001920700073
detect-time:  20.40539288520813
detect-time:  9.322262287139893
detect-time:  4.168564796447754
detect-time:  30.163954257965088
detect-time:  3.70723557472229
detect-time:  4.481033086776733
detect-time:  12.616742372512817
detect-time:  1.705559253692627
detect-time:  56.44028878211975
detect-time:  3.5832154750823975
detect-time:  18.579060554504395
detect-time:  27.307228565216064
detect-time:  2.2451329231262207
detect-time:  9.200062036514282
detect-time:  1.2917640209197998
detect-time:  51.577963113

detect-time:  9.863565683364868
detect-time:  2.849256992340088
detect-time:  3.6179094314575195
detect-time:  2.6464197635650635
detect-time:  2.115791082382202
detect-time:  14.981221914291382
detect-time:  2.376804828643799
detect-time:  6.647246360778809
detect-time:  8.345330953598022
detect-time:  19.96889567375183
detect-time:  3.5966522693634033
detect-time:  3.782965898513794
detect-time:  3.8347911834716797
detect-time:  8.415191888809204
detect-time:  4.6384429931640625
detect-time:  2.0079569816589355
detect-time:  1.8862035274505615
detect-time:  3.9686520099639893
detect-time:  0.924868106842041
detect-time:  2.276946783065796
detect-time:  14.516675233840942
detect-time:  1.7546327114105225
detect-time:  2.0207104682922363
detect-time:  7.019493579864502
detect-time:  1.5836544036865234
detect-time:  7.949815988540649
detect-time:  1.2530996799468994
detect-time:  1.4152705669403076
detect-time:  0.8777637481689453
detect-time:  1.0112392902374268
detect-time:  6.2538788

detect-time:  1.1920068264007568
detect-time:  1.163268804550171
detect-time:  1.8702104091644287
detect-time:  2.683262348175049
detect-time:  1.0748045444488525
detect-time:  2.337435722351074
detect-time:  0.9329187870025635
detect-time:  1.0908598899841309
detect-time:  3.295132637023926
detect-time:  1.1659882068634033
detect-time:  1.1872467994689941
idx:  500  ;neg_idx:  28285  ;pos_idx:  23888  ;par_idx:  15263
3700.704855442047
detect-time:  4.025202035903931
detect-time:  3.2145702838897705
detect-time:  2.573680877685547
detect-time:  1.1918690204620361
detect-time:  11.051772117614746
detect-time:  1.1748878955841064
detect-time:  1.2969915866851807
detect-time:  1.1258604526519775
detect-time:  1.414808750152588
detect-time:  0.9905431270599365
detect-time:  7.4704930782318115
detect-time:  0.9489476680755615
detect-time:  1.4290673732757568
detect-time:  51.5095694065094
detect-time:  0.9660067558288574
detect-time:  0.658883810043335
detect-time:  1.5136046409606934
dete

detect-time:  1.1309864521026611
detect-time:  1.7055888175964355
detect-time:  4.675097227096558
detect-time:  8.866142272949219
detect-time:  1.9331824779510498
detect-time:  22.632046699523926
detect-time:  2.3833045959472656
detect-time:  2.253506898880005
detect-time:  27.88076615333557
detect-time:  3.513686180114746
detect-time:  69.78800654411316
detect-time:  6.033599853515625
detect-time:  5.401303052902222
detect-time:  2.967163562774658
detect-time:  4.173688650131226
detect-time:  15.92017936706543
detect-time:  1.3300416469573975
detect-time:  3.7061479091644287
detect-time:  20.486063241958618
detect-time:  2.518256425857544
detect-time:  6.61217737197876
detect-time:  2.5599167346954346
detect-time:  1.2782902717590332
detect-time:  3.062201499938965
detect-time:  4.032702207565308
detect-time:  2.2741923332214355
detect-time:  1.814213752746582
detect-time:  2.9750335216522217
detect-time:  8.100398063659668
detect-time:  2.9039206504821777
detect-time:  5.851168394088

detect-time:  3.446174144744873
detect-time:  1.5747082233428955
detect-time:  1.5328962802886963
detect-time:  0.809014081954956
detect-time:  1.4841725826263428
detect-time:  2.972048759460449
detect-time:  1.2229084968566895
detect-time:  1.1608257293701172
detect-time:  4.036506414413452
detect-time:  48.39262533187866
detect-time:  2.0244224071502686
detect-time:  2.4528164863586426
detect-time:  1.622704267501831
detect-time:  7.235936641693115
detect-time:  2.799562692642212
detect-time:  1.0480148792266846
detect-time:  0.7868850231170654
detect-time:  1.5672385692596436
detect-time:  1.3589990139007568
idx:  1000  ;neg_idx:  53962  ;pos_idx:  39382  ;par_idx:  24495
6904.955500364304
detect-time:  28.534798622131348
detect-time:  3.3842129707336426
detect-time:  14.868731498718262
detect-time:  5.580298900604248
detect-time:  5.784693479537964
detect-time:  4.8650431632995605
detect-time:  5.177894353866577
detect-time:  5.582091569900513
detect-time:  4.0909693241119385
detec

...

detect-time:  2.2683305740356445
detect-time:  1.059983491897583
detect-time:  9.163993120193481
detect-time:  1.418799877166748
detect-time:  1.293203592300415
detect-time:  3.005725622177124
detect-time:  7.676119327545166
detect-time:  4.2541704177856445
detect-time:  1.657029628753662
detect-time:  4.386196136474609
detect-time:  1.1102368831634521
detect-time:  1.4315054416656494
detect-time:  1.8119444847106934
detect-time:  0.7722878456115723
detect-time:  1.3375639915466309
detect-time:  1.1283819675445557
detect-time:  1.6167235374450684
detect-time:  1.233483076095581
detect-time:  1.1452391147613525
detect-time:  1.691453218460083
detect-time:  1.4000177383422852
detect-time:  60.20553731918335
detect-time:  1.0699656009674072
detect-time:  3.4720301628112793
detect-time:  3.17146372795105
detect-time:  3.349195718765259
detect-time:  0.7390520572662354
detect-time:  2.6879470348358154
detect-time:  0.8496448993682861
detect-time:  0.800520658493042
detect-time:  2.016656637

detect-time:  18.19037628173828
detect-time:  2.178647756576538
detect-time:  144.3694772720337
detect-time:  3.176060676574707
detect-time:  2.626582622528076
detect-time:  4.599747180938721
detect-time:  2.0688884258270264
detect-time:  1.6209473609924316
detect-time:  6.4431281089782715
detect-time:  1.3150031566619873
detect-time:  7.790147304534912
detect-time:  2.5032248497009277
detect-time:  7.0265092849731445
detect-time:  0.8633790016174316
detect-time:  2.102295160293579
detect-time:  0.6953637599945068
detect-time:  0.9979758262634277
detect-time:  3.9589812755584717
detect-time:  2.408207654953003
detect-time:  4.415486812591553
detect-time:  4.04094672203064
detect-time:  6.612881422042847
detect-time:  1.4394659996032715
detect-time:  1.2824809551239014
detect-time:  15.266719102859497
detect-time:  1.8345935344696045
detect-time:  2.1619105339050293
idx:  12100  ;neg_idx:  587441  ;pos_idx:  213034  ;par_idx:  134944
76033.48500680923
detect-time:  26.182720184326172
de

detect-time:  1.0961992740631104
detect-time:  12.587425947189331
detect-time:  1.4545702934265137
detect-time:  1.2069852352142334
detect-time:  0.9937379360198975
detect-time:  2.8905811309814453
detect-time:  2.341975450515747
detect-time:  7.544491529464722
detect-time:  4.485046863555908
detect-time:  1.5769031047821045
detect-time:  20.228943824768066
detect-time:  1.7940878868103027
detect-time:  0.6204311847686768
detect-time:  5.677802801132202
detect-time:  3.258850574493408
detect-time:  1.0477802753448486
detect-time:  0.7918097972869873
detect-time:  7.659641981124878
detect-time:  6.157050848007202
detect-time:  0.9121809005737305
detect-time:  1.5776007175445557
detect-time:  8.686853170394897
detect-time:  1.4671440124511719
detect-time:  17.21747136116028
detect-time:  2.383202075958252
detect-time:  1.0536994934082031
detect-time:  2.211480140686035
detect-time:  4.610048294067383
detect-time:  2.7106130123138428
detect-time:  10.611396789550781
detect-time:  19.80056

detect-time:  1.3772432804107666
detect-time:  3.324416399002075
detect-time:  0.8949398994445801
detect-time:  3.7693674564361572
detect-time:  2.180417537689209
detect-time:  1.8865869045257568
detect-time:  2.0325074195861816
detect-time:  1.8006489276885986
detect-time:  1.1925592422485352
detect-time:  1.0565450191497803
detect-time:  1.926713466644287
detect-time:  2.485793352127075
detect-time:  0.9304172992706299
detect-time:  1.6183006763458252
detect-time:  2.656500816345215
detect-time:  1.0502829551696777
detect-time:  1.6457023620605469
detect-time:  1.754225730895996
detect-time:  0.9205200672149658
detect-time:  2.587986946105957
detect-time:  1.88319730758667
detect-time:  4.204369783401489
detect-time:  2.0929484367370605
detect-time:  0.9082145690917969
detect-time:  2.566946268081665
detect-time:  1.808825969696045
detect-time:  3.105999231338501
detect-time:  3.6727542877197266
detect-time:  1.517176628112793
detect-time:  2.2090892791748047
detect-time:  1.72249722

detect-time:  6.374818325042725
detect-time:  6.168111562728882
detect-time:  4.122239828109741
detect-time:  1.7220735549926758
detect-time:  1.6427891254425049
detect-time:  1.0300300121307373
detect-time:  1.838047981262207
detect-time:  1.4348270893096924
detect-time:  1.6451377868652344
detect-time:  90.32842898368835
detect-time:  1.4839568138122559
detect-time:  3.2706024646759033
detect-time:  26.220864295959473
detect-time:  0.7244131565093994
detect-time:  1.3774242401123047
detect-time:  1.097508192062378
detect-time:  1.8021726608276367
detect-time:  1.584235429763794
detect-time:  1.0218961238861084
detect-time:  18.304134368896484
detect-time:  1.0318398475646973
detect-time:  2.2792856693267822
detect-time:  0.9106574058532715
detect-time:  0.9971680641174316
detect-time:  2.043104648590088
detect-time:  9.645040273666382
detect-time:  1.4440670013427734
detect-time:  2.5051238536834717
detect-time:  0.8148479461669922
detect-time:  6.308077573776245
detect-time:  1.7764

# gen_O48_landmark_data

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
from core.tool import IoU,flip
import numpy as np
import random
from numpy.random import randint
import cv2
import os
import time

def main():
    
    f4 = open(os.path.join(save_dir, 'land_%d.txt'%(img_size)), 'w')

    with open(pic_spilt_dir) as filenames:
        land_idx=0
        for line in filenames:
            img_list=[]
            mark_list=[]
            line=line.strip().split(' ')
            img=cv2.imread(os.path.join(lfw_dir,line[0].replace("\\","/")))
            box=(line[1],line[2],line[3],line[4])
            box=[int(_) for _ in box]
            #format of box is [x,x+w,y,y+h]
            height,weight,channel=img.shape
            landmark=np.zeros((5,2))
            for i in range(5):
                mark=(float(line[5+2*i]),float(line[5+2*i+1]))
                landmark[i]=mark

            facemark=np.zeros((5,2))
            for i in range(5):
                mark=((landmark[i][0]-box[0])/(box[1]-box[0]),(landmark[i][1]-box[2])/(box[3]-box[2]))
                facemark[i]=mark
            img_list.append(cv2.resize(img[box[2]:box[3],box[0]:box[1]], (img_size, img_size)))  
            mark_list.append(facemark.reshape(10))

            box_=[box[0],box[2],box[1],box[3]]
            #format of box is [x,y,x+w,y+h]      
            x1,y1,x2,y2=box_
            w=x2-x1+1
            h=y2-y1+1

            if((x1<0)|(y1<0)|(max(w,h)<40)|(min(w,h)<=5)): 
                continue          
            num=30
            while(num):

                size=randint(np.floor(0.8*min(w,h)),np.ceil(1.25*max(w,h))+1)

                delta_w = randint(-w * 0.2, w * 0.2 + 1)
                delta_h = randint(-h * 0.2, h * 0.2 + 1)
                # random face box
                nx1 = int(max(x1 + w / 2 + delta_w - size / 2, 0))
                ny1 = int(max(y1 + h / 2 + delta_h - size / 2, 0))
                nx2 = nx1 + size
                ny2 = ny1 + size 

                if( nx2 > weight | ny2 > height):
                    continue               

                _box=[x1,y1,w,h]
                _box=np.array(_box).reshape(1,-1)     
                if(IoU(np.array([nx1,ny1,nx2,ny2]),_box)>0.65): 
                    facemark=np.zeros((5,2))
                    for i in range(5):
                        mark=((landmark[i][0]-nx1)/size,(landmark[i][1]-ny1)/size)
                        facemark[i]=mark  
                    img_list.append(cv2.resize(img[ny1:ny2,nx1:nx2,:], (img_size, img_size)))  
                    mark_list.append(facemark.reshape(10))

                    #mirro
                    mirro_mark=facemark.copy()
                    if(random.choice([0,1])):
                        img1,mirro_mark=flip(img[ny1:ny2,nx1:nx2,:],mirro_mark)
                        img_list.append(cv2.resize(img1, (img_size, img_size)))  
                        mark_list.append(mirro_mark.reshape(10))  

                    num=num-1
            for i in range(len(img_list)):

                if np.sum(np.where(mark_list[i] <= 0, 1, 0)) > 0:
                    continue

                if np.sum(np.where(mark_list[i] >= 1, 1, 0)) > 0:
                    continue

                cv2.imwrite(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)),img_list[i])
                mark=[str(_)for _ in mark_list[i]]
                f4.write(os.path.join(landmark_dir,'land_%d.jpg'%(land_idx)) +' -2 '+' '.join(mark)+'\n')
                land_idx=land_idx+1
            
        print("pics all done,land_pics %d in total"%(land_idx))
        
    f4.close()    

if __name__=="__main__":

    img_size=48
    
    #change img_size to P=12 R=24 O=48 net
    
    begin=time.time()
    
    lfw_dir="/home/dell/Desktop/prepared_data/train"
    pic_spilt_dir="/home/dell/Desktop/prepared_data/train/trainImageList.txt"
    landmark_dir="/home/dell/Desktop/prepared_data/DATA/%d/landmark"%(img_size)
    save_dir="/home/dell/Desktop/prepared_data/DATA/%d"%(img_size)

    if not os.path.exists(landmark_dir):
        os.makedirs(landmark_dir)    

    main()
    
    print(time.time()-begin)
    
# pics all done,land_pics 440756 in total
# 131.18922185897827

# gen_O48_tfrecord

In [1]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf 
import cv2
import random
import time
import os

def main():
    
    t_time=time.time()   
    for index,term in enumerate(terms):
        num=0
        print("%s start"%(term))
        with tf.python_io.TFRecordWriter(os.path.join(base_dir,"DATA/%d/%s_train.tfrecords"%(img_size,term))) as writer:
            with open(os.path.join(base_dir,"DATA/%d/%s.txt"%(img_size,term))) as readlines:
                readlines=[line.strip().split(' ') for line in readlines]
                random.shuffle(readlines)
                for i,line in enumerate(readlines):
                    if(num%10000==0):
                        print(i,time.time()-t_time)
                        t_time=time.time()
                    #if windows
                    img=cv2.imread(line[0])
                    if(img is None):
                        continue
                    img_raw = img.tobytes()
                    label=int(line[1])
                    roi=[0.0]*4               
                    landmark=[0.0]*10
                    if(len(line)==6):    
                        roi=[float(_) for _ in line[2:6]]   
                    if(len(line)==12):
                        landmark=[float(_) for _ in line[2:12]]                  
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "roi": tf.train.Feature(float_list=tf.train.FloatList(value=roi)),
                        "landmark": tf.train.Feature(float_list=tf.train.FloatList(value=landmark)),
                    }))
                    writer.write(example.SerializeToString())  #序列化为字符串  
                    num+=1
                    if(num==base*scale[index]):
                        print("%s finish"%(term))
                        break

if __name__=="__main__":
    
    img_size=48
    #change img_size to P=12 R=24 O=48 net
    terms=['neg_%d'%(img_size),'pos_%d'%(img_size),'par_%d'%(img_size),'land_%d'%(img_size)]
    scale=[3,1,1,2]
    
    base_dir="/home/dell/Desktop/prepared_data"
    
    #set base number of pos_pic    
    base=140000

    begin=time.time()

    main()
    
    print(time.time()-begin)
# 7471.2840230464935

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


neg_48 start
0 1.4879889488220215
10000 73.11700987815857
20000 75.49641251564026
30000 74.30164861679077
40000 71.60170102119446
50000 75.5090401172638
60000 73.68515753746033
70000 72.83425426483154
80000 75.05076313018799
90000 76.40816116333008
100000 72.98926568031311
110000 73.4943494796753
120000 73.88508105278015
130000 73.91212701797485
140000 73.30549192428589
150000 72.57569551467896
160000 73.55272626876831
170000 75.69970631599426
180000 74.03456711769104
190000 76.74130392074585
200000 73.52062940597534
210000 72.12152218818665
220000 75.75695872306824
230000 73.66633462905884
240000 75.42765212059021
250000 74.75302743911743
260000 74.68276810646057
270000 72.89439916610718
280000 76.47499561309814
290000 72.11877346038818
300000 73.87893772125244
310000 76.4434506893158
320000 72.98320174217224
330000 74.62772154808044
340000 73.96046042442322
350000 71.4073805809021
360000 74.20933723449707
370000 71.01559519767761
380000 73.80684971809387
390000 72.90676593780518
4000

# train_O48_model

In [1]:
# -*- coding: utf-8 -*-
"""
@author: friedhelm

"""
import tensorflow as tf
import time
from core.tool import read_multi_tfrecords,image_color_distort
from core.MTCNN_model import Pnet_model,Rnet_model,Onet_model
from train.train_tool import label_los,roi_los,landmark_los,cal_accuracy
import os

def train(image,label,roi,landmark,model,model_name):
    
    _label, _roi ,_landmark=model(image,batch)
    
    with tf.name_scope('output'):
        _label=tf.squeeze(_label,name='label')
        _roi=tf.squeeze(_roi,name='roi')
        _landmark=tf.squeeze(_landmark,name='landmark')
        
    _label_los=label_los(_label,label)
    _box_los=roi_los(label,_roi,roi)    
    _landmark_los=landmark_los(label,_landmark,landmark)
    
    function_loss=_label_los*ratio[0]+_box_los*ratio[1]+_landmark_los*ratio[2]

    tf.add_to_collection("loss", function_loss)
    loss_all=tf.get_collection('loss')
    
    with tf.name_scope('loss'):
        loss=tf.reduce_sum(loss_all)
        tf.summary.scalar('loss',loss) 

    opt=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        train_accuracy,truth_accuracy,false_accuracy=cal_accuracy(_label,label)
        tf.summary.scalar('accuracy',train_accuracy) 
        tf.summary.scalar('ture_accuracy',truth_accuracy) 
        tf.summary.scalar('false_accuracy',false_accuracy) 
        
    saver=tf.train.Saver(max_to_keep=10)
    merged=tf.summary.merge_all() 
    
    images,labels,rois,landmarks=read_multi_tfrecords(addr,batch_size,img_size)   
    images=image_color_distort(images)
    
    with tf.Session() as sess:
        sess.run((tf.global_variables_initializer(),
                  tf.local_variables_initializer()))
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess,coord=coord)
        image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
        
        writer_train=tf.summary.FileWriter(os.path.join(base_dir,"model/%s/"%(model_name)),sess.graph)
        try:
            
            for i in range(1,train_step):
                
                image_batch,label_batch,roi_batch,landmark_batch=sess.run([images,labels,rois,landmarks])
                
                sess.run(opt,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                if(i%100==0):
                    summary=sess.run(merged,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})
                    writer_train.add_summary(summary,i) 
                if(i%1000==0):
                    print('次数',i)    
                    print('train_accuracy',sess.run(train_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('truth_accuracy',sess.run(truth_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('false_accuracy',sess.run(false_accuracy,feed_dict={image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))
                    print('loss',sess.run(loss,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch})) 
                    print('_label_los',sess.run(_label_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_box_los',sess.run(_box_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('_landmark_los',sess.run(_landmark_los,{image:image_batch,label:label_batch,roi:roi_batch,landmark:landmark_batch}))                     
                    print('time',time.time()-begin)
                    if(i%10000==0):
                        saver.save(sess,os.path.join(base_dir,"model/%s/%s.ckpt"%(model_name,model_name)),global_step=i)
        except  tf.errors.OutOfRangeError:
            print("finished")
        finally:
            coord.request_stop()
            writer_train.close()
        coord.join(threads)
    
def main(model):
    
    with tf.name_scope('input'):
        image=tf.placeholder(tf.float32,[batch,img_size,img_size,3],name='image')
        label=tf.placeholder(tf.int32,[batch],name='label')
        roi=tf.placeholder(tf.float32,[batch,4],name='roi')
        landmark = tf.placeholder(tf.float32,[batch,10],name='landmark')  

    train(image,label,roi,landmark,model,model_name)

if __name__=='__main__':
    
    base_dir="/home/dell/Desktop/prepared_data"
    img_size=48
    batch=448
    batch_size=[192,64,64,128]
    
    addr=[os.path.join(base_dir,"DATA/%d/neg_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/pos_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/par_%d_train.tfrecords"%(img_size,img_size)),
          os.path.join(base_dir,"DATA/%d/land_%d_train.tfrecords"%(img_size,img_size))]  

    model=Onet_model
    model_name="Onet_model"    
    train_step=100001
    learning_rate=0.001
    
    save_model_path=os.path.join(base_dir,"model/%s"%(model_name))
    
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path) 
        
    if(model_name=="Onet_model"):
        ratio=[1,0.5,1]
    else:
        ratio=[1,0.5,0.5]
    

    begin=time.time()        
    main(model)
    
# tensorboard --logdir=/home/dell/Desktop/prepared_data/model/Onet_model/

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


次数 1000
train_accuracy 0.9296875
truth_accuracy 0.828125
false_accuracy 0.9635417
loss 0.73335916
_label_los 0.2416755
_box_los 0.025982095
_landmark_los 0.018345997
time 880.8781342506409
次数 2000
train_accuracy 0.9296875
truth_accuracy 0.734375
false_accuracy 0.9947917
loss 0.51390535
_label_los 0.23808137
_box_los 0.01949503
_landmark_los 0.008433127
time 1739.545506477356
次数 3000
train_accuracy 0.953125
truth_accuracy 0.859375
false_accuracy 0.984375
loss 0.36678568
_label_los 0.19664805
_box_los 0.02026699
_landmark_los 0.0058402386
time 2591.9997260570526
次数 4000
train_accuracy 0.9765625
truth_accuracy 0.96875
false_accuracy 0.9791667
loss 0.21766217
_label_los 0.10380554
_box_los 0.017944306
_landmark_los 0.004597005
time 3443.0781989097595
次数 5000
train_accuracy 0.97265625
truth_accuracy 0.953125
false_accuracy 0.9791667
loss 0.18533657
_label_los 0.10044798
_box_los 0.018655831
_landmark_los 0.0048455573
time 4293.740399837494
次数 6000
train_accuracy 0.95703125
truth_accuracy 0.

train_accuracy 0.9765625
truth_accuracy 0.921875
false_accuracy 0.9947917
loss 0.13004187
_label_los 0.082896896
_box_los 0.017298413
_landmark_los 0.0028188874
time 37192.13430953026
次数 45000
train_accuracy 0.98046875
truth_accuracy 0.9375
false_accuracy 0.9947917
loss 0.12719646
_label_los 0.08165245
_box_los 0.013145501
_landmark_los 0.0033049977
time 38032.560611248016
次数 46000
train_accuracy 0.97265625
truth_accuracy 0.921875
false_accuracy 0.9895833
loss 0.16488877
_label_los 0.11771254
_box_los 0.014554687
_landmark_los 0.004126523
time 38872.64363193512
次数 47000
train_accuracy 0.9765625
truth_accuracy 0.9375
false_accuracy 0.9895833
loss 0.14380412
_label_los 0.097587064
_box_los 0.016497642
_landmark_los 0.0023925267
time 39712.43196797371
次数 48000
train_accuracy 0.97265625
truth_accuracy 0.921875
false_accuracy 0.9895833
loss 0.18309972
_label_los 0.13691789
_box_los 0.0144984145
_landmark_los 0.0028938171
time 40552.471458911896
次数 49000
train_accuracy 0.97265625
truth_accur

KeyboardInterrupt: 